In [ ]:
# Persons: Id (PK), PersonCode, FirstName, MiddleName, LastName, Email, DOB, DOJ, ContactNumber, EmergencyNumber, BloodGroup, MaritalStatus, Gender, DepartmentId, DesignationId, RoleId, OrganizationId, LoginType, Password, ReportingPerson, IsActive, LastWorkingDay, IsPolicyConsent, PolicyConsentDate, ProfilePicture, CreatedDate, CreatedBy, Otp, QuickOtp, IsCaptured, IsTrained

# PersonDocument: Id (PK), PersonId, DocumentNumber, ExpiryDate, Remarks, DocumentType, Path, Description
#   - PersonId → Persons.Id (foreign key)

# PersonEducations: Id (PK), PersonId, Degree, PassingYear, Grade, Institution, Address
#   - PersonId → Persons.Id (foreign key)

# PersonExperiences: Id (PK), PersonId, CompanyName, StartDate, EndDate
#   - PersonId → Persons.Id (foreign key)

# PersonFamilyDetails: Id (PK), PersonId, RelationshipType, FirstName, MiddleName, LastName, DOB, ContactNumber, BloodGroup, Gender
 



In [4]:
import pyodbc
import re
import os
from sentence_transformers import SentenceTransformer, util
from langchain_groq import ChatGroq
import speech_recognition as sr

api_key = "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"
llm = ChatGroq(api_key=api_key, model_name="llama3-70b-8192", streaming=False)
model = SentenceTransformer('all-MiniLM-L6-v2')

def load_schema_text(file_path: str) -> str:
    with open(file_path, 'r', encoding="utf-8") as f:
        return f.read()

def parse_schema_to_table_blocks(schema_text: str) -> dict:
    blocks = schema_text.strip().split('\n\n')
    table_blocks = {}
    for block in blocks:
        lines = block.strip().splitlines()
        if not lines:
            continue
        match = re.match(r"^([A-Za-z0-9_]+):", lines[0])
        if match:
            table_name = match.group(1)
            table_blocks[table_name] = block.strip()
    return table_blocks

def connect_to_ssms(server, database, username=None, password=None, trusted_connection=True):
    try:
        if trusted_connection:
            conn_str = (
                f'DRIVER={{ODBC Driver 17 for SQL Server}};'
                f'SERVER={server};'
                f'DATABASE={database};'
                'Trusted_Connection=yes;'
            )
        else:
            conn_str = (
                f'DRIVER={{ODBC Driver 17 for SQL Server}};'
                f'SERVER={server};'
                f'DATABASE={database};'
                f'UID={username};'
                f'PWD={password};'
            )
        conn = pyodbc.connect(conn_str)
        print("Database connection successful")
        return conn
    except Exception as e:
        print(" Database connection failed:", str(e))
        return None

schema_text = load_schema_text("schema_output_llm.txt")
schema = parse_schema_to_table_blocks(schema_text)
table_names = list(schema.keys())
table_descriptions = list(schema.values())
schema_embeddings = model.encode(table_descriptions, convert_to_tensor=True)

def get_relevant_tables(question: str, top_n=3):
    question_embedding = model.encode(question, convert_to_tensor=True)
    similarities = util.cos_sim(question_embedding, schema_embeddings)[0]
    top_indices = similarities.argsort(descending=True)[:top_n]
    return [table_names[i] for i in top_indices]

def query_ssms_with_context(natural_question: str) -> str:
    try:
        natural_question, PersonId = [x.strip() for x in natural_question.split("|", 1)]
    except ValueError:
        return " Invalid input. Use format: 'question | PersonId'"
    relevant_tables = get_relevant_tables(natural_question)
    context_blocks = [schema[table] for table in relevant_tables if table in schema]
    schema_context = "\n\n".join(context_blocks)

    prompt = f"""
You are an expert SQL query generator. Based only on the tables and columns listed below, write a syntactically correct and semantically accurate SQL SELECT query that answers the given question.

### Schema:
{schema_context}

### User Question:
{natural_question}

❗"Only return the SQL query, nothing else."

### Important rules:
1. If the question involves any table not listed above, respond with: I can't help you with that request.
2. If the question involves any of these tables:
   PersonFamilyDetails, PersonExperiences, PersonEducations, PersonDocument,
   Attendances, AttendanceDetails
   then filter the results by: PersonId = '{PersonId}'
3. If the question involves the Persons table, then filter the results by: Id = '{PersonId}'
4. If the user asks about "date of birth", map it to column `dob` in the `Persons` table.
5. If the user asks about "date of joining", map it to column `doj` in the `Persons` table.


### Instructions:
- Only use the tables and columns provided in the schema above.
- Use JOINs only if needed and valid.
- Do NOT make up any table or column names.
- Do NOT explain anything. Return only the final SQL SELECT query.
- Output only valid SQL.
### SQL Query:
"""

    try:
        sql_query = llm.invoke(prompt).content.strip().replace("`", "")
        print("Generated SQL Query:", sql_query)
    except Exception as e:
        return f" LLM Error: {str(e)}"

    os.makedirs("logs", exist_ok=True)
    with open("logs/prompt_log.txt", "a", encoding="utf-8") as f:
        f.write(f"\n---\nPrompt:\n{prompt}\nSQL:\n{sql_query}\n")

    conn = connect_to_ssms(
        server= 'DESKTOP-MOM300C',
        database='SkyHR',
        trusted_connection=True
    )
    if conn is None:
        return " Failed to connect to the database."

    try:
        cursor = conn.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        result = [dict(zip(columns, row)) for row in rows]
        print("SQL Query executed successfully. Rows returned:", result)
        if result:
    # Join all rows into one big string
            all_answers = []
            for row in result:
                row_str = ", ".join(f"{k}: {v}" for k, v in row.items())
                all_answers.append(f"[{row_str}]")
            answer = " | ".join(all_answers)  # separate rows with pipe or newline
            return f"Answer: {answer}"
        else:
            return "No data found for the provided ID."

    except Exception as e:
        return f"Error executing SQL:\n{sql_query}\n\nError: {str(e)}"
    finally:
        conn.close()

if __name__ == "__main__":

    r = sr.Recognizer()

    with sr.Microphone() as source:
        print("Speak something (you have 10 seconds)...")
        try:
            audio = r.listen(source, phrase_time_limit=10, timeout=10)
            user_query = r.recognize_google(audio)
            print("You said:", user_query)
        except sr.WaitTimeoutError:
            # No speech detected within 10 seconds
            print("No speech detected within 10 seconds.")
            user_query = input("Please type your query: ")
        except sr.UnknownValueError:
            # Speech unintelligible
            print("Could not understand audio.")
            user_query = input("Please type your query: ")
        except sr.RequestError as e:
            print(f"Speech recognition service error: {e}")
            user_query = input("Please type your query: ")

    print("Final input:", user_query)


    # user_query = "give me family details"  
    PersonId = 'AFFBC428-C29B-44FA-82F0-FC764CF57A5A'
    combined_input = f"{user_query} | {PersonId}"
    
    response = query_ssms_with_context(combined_input)
    print("response",response)
    
    prompt = (
    f"Convert the following structured information into a human-readable format:\n\n"
    f"Do not include any headings, explanations, or extra text. Just return the final sentence:\n\n"
    f"{response}\n\n"
    f"Make sure the result sounds like a natural sentence."
)

    human_readable_response = llm.invoke(prompt)
    print("human_readable_response",human_readable_response.content)


Speak something (you have 10 seconds)...
No speech detected within 10 seconds.
Final input: email
Generated SQL Query: SELECT Email 
FROM Persons 
WHERE Id = 'AFFBC428-C29B-44FA-82F0-FC764CF57A5A';
Database connection successful
SQL Query executed successfully. Rows returned: [{'Email': 'priyank.patel@skyttus.com'}]
response Answer: [Email: priyank.patel@skyttus.com]
human_readable_response The answer is priyank.patel@skyttus.com.


In [10]:
import pyodbc
import re
import os
from sentence_transformers import SentenceTransformer, util
from langchain_groq import ChatGroq

api_key = "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"
llm = ChatGroq(api_key=api_key, model_name="llama3-70b-8192", streaming=False)
model = SentenceTransformer('all-MiniLM-L6-v2')

def load_schema_text(file_path: str) -> str:
    with open(file_path, 'r', encoding="utf-8") as f:
        return f.read()

def parse_schema_to_table_blocks(schema_text: str) -> dict:
    blocks = schema_text.strip().split('\n\n')
    table_blocks = {}
    for block in blocks:
        lines = block.strip().splitlines()
        if not lines:
            continue
        match = re.match(r"^([A-Za-z0-9_]+):", lines[0])
        if match:
            table_name = match.group(1)
            table_blocks[table_name] = block.strip()
    return table_blocks

def connect_to_ssms(server, database, username=None, password=None, trusted_connection=True):
    try:
        if trusted_connection:
            conn_str = (
                f'DRIVER={{ODBC Driver 17 for SQL Server}};'
                f'SERVER={server};'
                f'DATABASE={database};'
                'Trusted_Connection=yes;'
            )
        else:
            conn_str = (
                f'DRIVER={{ODBC Driver 17 for SQL Server}};'
                f'SERVER={server};'
                f'DATABASE={database};'
                f'UID={username};'
                f'PWD={password};'
            )
        conn = pyodbc.connect(conn_str)
        print("Database connection successful")
        return conn
    except Exception as e:
        print(" Database connection failed:", str(e))
        return None

schema_text = load_schema_text("company_policy.txt")
schema = parse_schema_to_table_blocks(schema_text)
table_names = list(schema.keys())
table_descriptions = list(schema.values())
schema_embeddings = model.encode(table_descriptions, convert_to_tensor=True)

def get_relevant_tables(question: str, top_n=3):
    question_embedding = model.encode(question, convert_to_tensor=True)
    similarities = util.cos_sim(question_embedding, schema_embeddings)[0]
    top_indices = similarities.argsort(descending=True)[:top_n]
    return [table_names[i] for i in top_indices]

def query_ssms_with_context(natural_question: str) -> str:
    try:
        natural_question, PersonId = [x.strip() for x in natural_question.split("|", 1)]
    except ValueError:
        return " Invalid input. Use format: 'question | PersonId'"
    relevant_tables = get_relevant_tables(natural_question)
    context_blocks = [schema[table] for table in relevant_tables if table in schema]
    schema_context = "\n\n".join(context_blocks)

    prompt = f"""
You are an expert SQL Server (T-SQL) query generator. Based only on the tables and columns listed below, write a syntactically correct and semantically accurate SQL SELECT query that answers the given question.

### Schema:
{schema_context}

### User Question:
{natural_question}

❗"Only return the SQL query, nothing else."

### Important rule:
- Only generate a query if the question involves **PolicyType**, **Organizations**, or **CompanyPolicy** tables.  
  If it involves any other table, respond with: I can't help you with that request.

### Instructions:
- Only use the tables and columns provided in the schema above.
- Only generate queries using **Microsoft SQL Server (T-SQL)** syntax. Do NOT use MySQL or PostgreSQL syntax.
- Use JOINs only if needed and valid.
- Do NOT make up any table or column names.
- Do NOT use insert, update, delete, alter queries.
- Do NOT explain anything. Return only the final SQL SELECT query.
- Output only valid T-SQL, unless restricted by above rules.

### SQL Query:
"""


    try:
        sql_query = llm.invoke(prompt).content.strip().replace("`", "")
    except Exception as e:
        return f" LLM Error: {str(e)}"

    os.makedirs("logs", exist_ok=True)
    with open("logs/prompt_log.txt", "a", encoding="utf-8") as f:
        f.write(f"\n---\nPrompt:\n{prompt}\nSQL:\n{sql_query}\n")

    conn = connect_to_ssms(
        server= 'DESKTOP-MOM300C',
        database='SkyHR',
        trusted_connection=True
    )
    if conn is None:
        return " Failed to connect to the database."

    try:
        cursor = conn.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        result = [dict(zip(columns, row)) for row in rows]
        print("result", result)
        if result:
            answers = []
            for row in result:
                row_text = ", ".join(f"{k}: {v}" for k, v in row.items())
                answers.append(row_text)
            return "Answers:\n" + "\n".join(answers)
        else:
            return "No data found for the provided ID."

    except Exception as e:
        return f"Error executing SQL:\n{sql_query}\n\nError: {str(e)}"
    finally:
        conn.close()

if __name__ == "__main__":

    user_query = "organization name"
    PersonId = '0A6EFDC5-A31B-47F3-AEDE-B149B845AA69'
    combined_input = f"{user_query} | {PersonId}"
    response = query_ssms_with_context(combined_input)
    print("response",response)
    
    prompt = (
    f"Convert the following structured information into a human-readable format:\n\n"
    f"{response}\n\n"
    f"Make sure the result sounds like a natural sentence."
)

    human_readable_response = llm.invoke(prompt)
    print("human_readable_response",human_readable_response.content)


Database connection successful
result [{'Name': 'Skytus'}]
response Answers:
Name: Skytus
human_readable_response Here is the converted information in a human-readable format:

The answer is Skytus.


## tools and agents

In [29]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

ssms_query_tool = Tool(
    name="SSMSQueryTool",
    func=query_ssms_with_context,
    description="Use this tool to query the SQL Server database. Input should be a user question in plain English."
)

agent = initialize_agent(
    tools=[ssms_query_tool],
    llm=llm,
    # agent="zero-shot-react-description",
    AgentType="zero-shot-react-description",  # Decide Which Tool to Use Based on the User's Input
    verbose=True,
    handle_parsing_errors=True
)

if __name__ == "__main__":
    user_query = "give me family details"
    PersonId = '0A6EFDC5-A31B-47F3-AEDE-B149B845AA69'
    combined_input = f"{user_query} | {PersonId}"
    response = agent.run(combined_input)
    prompt = (
        f"Convert the following structured information into a human-readable format:\n\n"
        f"Only return the final sentence, without any preamble or explanation.\n\n"
        f"but do not include any person ID or unique identifiers. \n\n"
        f"{response}\n\n"
        f"Make sure the result sounds like a natural sentence."
    )

    human_readable_response = llm.invoke(prompt)
    print("human_readable_response",human_readable_response.content)



> Entering new AgentExecutor chain...
Thought: I need to query the database to retrieve family details associated with the given ID.

Action: SSMSQueryTool
Action Input: "Select * from FamilyDetails where FamilyID = '0A6EFDC5-A31B-47F3-AEDE-B149B845AA69'"

Observation:  Invalid input. Use format: 'question | PersonId'
Thought:It seems like I need to reformat the input to include the PersonId instead of FamilyId.

Action: SSMSQueryTool
Action Input: "Select * from Person where PersonId = '0A6EFDC5-A31B-47F3-AEDE-B149B845AA69'"

Observation:  Invalid input. Use format: 'question | PersonId'
Thought:It seems like I'm stuck in an infinite loop!

Let me try to reformulate the input to match the expected format.

Action: SSMSQueryTool
Action Input: "What are the family details of the person with id 0A6EFDC5-A31B-47F3-AEDE-B149B845AA69? | 0A6EFDC5-A31B-47F3-AEDE-B149B845AA69"
Database connection successful

Observation: Answer: Id: 53C2C93B-C87A-4ACC-ACF5-08DC591E3DEE, PersonId: 0A6EFDC5-A3

In [32]:
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import initialize_agent
from langchain_groq import ChatGroq
from urllib.parse import quote_plus

# 1. Create database URI
params = quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-MOM300C;"
    "DATABASE=SkyHR;"
    "Trusted_Connection=yes;"
)
db_url = f"mssql+pyodbc:///?odbc_connect={params}"

api_key = "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"
llm = ChatGroq(api_key=api_key, model_name="llama3-70b-8192", streaming=False)

sql_db = SQLDatabase.from_uri(
    db_url,
    include_tables=["PolicyType", "Organizations", "CompanyPolicy"]
)

sql_toolkit = SQLDatabaseToolkit(db=sql_db, llm=llm)
sql_tools = sql_toolkit.get_tools()

agent = initialize_agent(
    tools=sql_tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    handle_parsing_errors=True,
    verbose=False  
)

input_query = "give me description for policy 'Employees will submit their cellphones to our HR' in companyPolicy table"
response = agent.invoke({"input": input_query}) 

print(response["output"])


We take one more change in our company policy. All employees will submit their cellphones to our HR. I hope you all cooperate
